In [ ]:
from edgar import set_identity, Company

# Use your name and email (required by SEC)
set_identity("John Doe john.doe@company.com")

# 2. Initialize the Company object for Tesla
tsla = Company("TSLA")

# --- OPTION A: Get the April Announcement (Form 8-K) ---
# This contains the press release from April 1st roughly regarding the profit.
announcements = tsla.get_filings(form="8-K").filter(date="2013-04-01:2013-04-30")

if len(announcements) > 0:
    april_announcement = announcements[0]
    print(f"Found Announcement: {april_announcement}")
    print(f"Filed on: {april_announcement.filing_date}")
    # View the content in the terminal or notebook
    # april_announcement.view()

# --- OPTION B: Get the Full Q1 Financial Report (Form 10-Q) ---
# Note: Even though results were Q1 (Jan-Mar), the report is filed in May.
quarterly_reports = tsla.get_filings(form="10-Q").filter(date="2013-05-01:2013-05-31")

if len(quarterly_reports) > 0:
    q1_report = quarterly_reports[0]
    print(f"\nFound Full Report: {q1_report}")
    print(f"Filed on: {q1_report.filing_date}")

    # Access the full text or HTML
    # report_html = q1_report.html()
    # report_text = q1_report.markdown()

    # You can also inspect the balance sheet or income statement directly
    # print(q1_report.financials.income_statement)
else:
    print("No 10-Q found in that date range.")


In [5]:
from edgar import Company, set_identity
import pandas as pd

# 1. SETUP: Set your SEC identity (User Agent)
# Replace with your actual name and email
set_identity("John Doe john.doe@company.com")

def get_tesla_2013_growth():
    # 2. FETCH: Get Tesla's filings
    tsla = Company("TSLA")

    # We filter for the 10-Q filed in May 2013, which covers the Q1 (Jan-Mar) period
    filings = tsla.get_filings(form="10-Q").filter(date="2013-05-01:2013-05-31")

    if not filings:
        print("Error: Filing not found.")
        return

    q1_report = filings[0]
    print(f"Processing Filing: {q1_report}")
    print(f"Filed Date: {q1_report.filing_date}")

    # 3. PARSE: Extract the Income Statement
    # This converts the XBRL data into a pandas DataFrame
    financials = q1_report.obj().financials
    income_statement = financials.income_statement
    # Clean up the index (row labels) for easier searching
    # income_statement.index = income_statement.index.str.strip()

    print("\n--- Raw Columns Found ---")
    print(income_statement.columns.tolist())

    # 4. EXTRACT: Locate Revenue and Net Income rows
    # Note: 2013 filings often used "Automotive sales" or "Total revenues"
    # We try multiple keys to ensure we find the right row.

    try:
        # Locating Revenue (Sales)
        if "Total revenues" in income_statement.index:
            revenue_row = income_statement.loc["Total revenues"]
        elif "Revenues" in income_statement.index:
            revenue_row = income_statement.loc["Revenues"]
        else:
            # Fallback: grab the first row if standard labels fail (risky but often works for top-line)
            revenue_row = income_statement.iloc[0]

        # Locating Net Income (Earnings)
        if "Net income (loss)" in income_statement.index:
            earnings_row = income_statement.loc["Net income (loss)"]
        elif "NetIncomeLoss" in income_statement.index:
            earnings_row = income_statement.loc["NetIncomeLoss"]
        else:
            # Use a robust search for "Net income" if exact match fails
            earnings_row = income_statement[income_statement.index.str.contains("Net income", case=False)].iloc[0]

        # 5. CALCULATE: Compare Q1 2013 to Q1 2012
        # The dataframe columns are usually dates.
        # Col 0 is typically the Current Quarter (2013-03-31)
        # Col 1 is typically the Prior Year Quarter (2012-03-31)

        current_sales = revenue_row.iloc[0]
        prior_sales = revenue_row.iloc[1]
        current_earnings = earnings_row.iloc[0]

        # Explicitly calculating Growth
        # Formula: ((Current - Prior) / Prior) * 100
        sales_growth_pct = ((current_sales - prior_sales) / prior_sales) * 100

        # 6. OUTPUT
        print(f"\n--- RESULTS (USD in Thousands) ---")
        print(f"Period: {income_statement.columns[0]} vs {income_statement.columns[1]}")
        print(f"Sales (Current Q):   ${current_sales:,.0f}")
        print(f"Sales (Prior Q):     ${prior_sales:,.0f}")
        print(f"Sales Growth YoY:    {sales_growth_pct:.2f}%")
        print(f"Net Income (Earn):   ${current_earnings:,.0f}")

    except Exception as e:
        print(f"\nError extracting data: {e}")
        print("This often happens if the specific row label changed in 2013.")
        print("Inspect 'income_statement' manually to find the exact row name.")

# Run the function
get_tesla_2013_growth()

Processing Filing: Filing(company='Tesla, Inc.', cik=1318605, form='10-Q', filing_date='2013-05-10', accession_no='0001193125-13-212354')
Filed Date: 2013-05-10

--- Raw Columns Found ---


AttributeError: 'function' object has no attribute 'columns'